<a href="https://colab.research.google.com/github/raulquinterog/Inteligencia-Artificial/blob/main/A1_4_Vino_Tinto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este proyecto encontré una base de datos en línea sobre muchos vinos diferentes y muchas características de tal. Aqui trate de encontrar alguna correlación entre las especificaciones como la acidez fija, cítrica, azucar residual, alcohol, densidad, etc. en la calidad en sí del vino.

In [5]:
import mlxtend
import pandas as pd

# Leer el archivo CSV en un DataFrame
df = pd.read_csv("A1.4 Vino Tinto.csv")

# Mostrar dimensiones del DataFrame
print(f"Dimensiones del DataFrame: {df.shape}")

# Mostrar las primeras 5 filas
df.head()

Dimensiones del DataFrame: (1599, 12)


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Primero importe las biblitecas que iba a necestiar, revisé las dimensiones y me di una buena idea de como estaban desplegadas.

In [12]:
from sklearn.model_selection import train_test_split

# Separar las características (X) y la variable objetivo (y)
X = df.drop(columns=['calidad'])  # eliminé la columna "calidad" para usar las demás como características
y = df['calidad']  # La variable objetivo es "calidad"

# Separar en datos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir dimensiones de los conjuntos
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")
print(f"Tamaño de y_train: {y_train.shape}")
print(f"Tamaño de y_test: {y_test.shape}")

# Verificar que la suma de filas de entrenamiento y prueba coincida con los datos originales
print(f"La suma de las filas de entrenamiento y prueba son: {len(X_train) + len(X_test)}")
if len(X_train) + len(X_test) == len(df):
    print("La suma coincide con los datos originales.")
else:
    print("La suma no coincide con los datos originales.")



Tamaño de X_train: (1279, 11)
Tamaño de X_test: (320, 11)
Tamaño de y_train: (1279,)
Tamaño de y_test: (320,)
La suma de las filas de entrenamiento y prueba son: 1599
La suma coincide con los datos originales.


En esta parte separé de manera aleatoria los datos usando la libreria sklearn.model_selection entre el 80% de los datos para los de entrenamiento y el otro 20% para las pruebas. Despúes verifiqué que se hizo la separación correctamente y todos los datos se estaban tomando en cuenta.

In [14]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# Crear el modelo de regresión lineal
modelo = LinearRegression()

# Configurar el selector de características hacia adelante
selector = SequentialFeatureSelector(
    modelo,
    k_features=(2, 8),  # Selecciona entre 2 y 8 características
    forward=True,  # Selección hacia adelante
    scoring='r2',  # Usa R cuadrada como métrica
    cv=10  # Validación cruzada con 10 folds
)

# Aplicar el selector en los datos de entrenamiento
selector.fit(X_train, y_train)

# Obtener las características seleccionadas
selected_features = list(selector.k_feature_names_)
print("Características seleccionadas:", selected_features)



Características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


Aqui aprendí a usar y configurar la metodología de selección hacia adelante de la libreria y la use para conseguir las variables que verdaderamente afectaban a los resultados. Aqui use la métrica de r2 para que se quede con la mejor variable y obtuve 7 características que son las que tienen algún efecto en la r2.

In [17]:
from sklearn.metrics import r2_score

# Filtrar X_train y X_test para usar solo las características seleccionadas
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Entrenar el modelo con las características seleccionadas
modelo.fit(X_train_selected, y_train)

# Hacer predicciones
y_pred = modelo.predict(X_test_selected)

# Calcular R² para evaluar el modelo
r2 = r2_score(y_test, y_pred)
print(f"R² del modelo con selección de características: {r2:.4f}")


R² del modelo con selección de características: 0.4013


Ya que obtuví las variables más relevantes del sistema, ahora puedo aplicar el modelo y calcular la r2 para ver que tan confiable es.

In [16]:
# Configurar el selector de características hacia atrás
selector_backward = SequentialFeatureSelector(
    modelo,
    k_features=(2, 5),  # Ahora probamos entre 2 y 5 características
    forward=False,  # Selección hacia atrás
    scoring='r2',
    cv=10
)

# Aplicar el selector en los datos de entrenamiento
selector_backward.fit(X_train, y_train)

# Obtener las características seleccionadas
selected_features_backward = list(selector_backward.k_feature_names_)
print("Características seleccionadas (hacia atrás):", selected_features_backward)


Características seleccionadas (hacia atrás): ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


Despúes verificamos con la selección hacia atrás si hay alguna mejora que podemos hacer en las variables seleccionadas. Es importante hacer esto porque aseguras que no estas usando una variable que no afecta a los resultados finales.

In [23]:
# Filtrar X_train y X_test para usar solo las características seleccionadas en la selección hacia atrás
X_train_backward = X_train[selected_features_backward]
X_test_backward = X_test[selected_features_backward]

# Entrenar el modelo con las nuevas características seleccionadas
modelo.fit(X_train_backward, y_train)

# Hacer predicciones
y_pred_backward = modelo.predict(X_test_backward)

# Calcular R² para evaluar el modelo
r2_backward = r2_score(y_test, y_pred_backward)
print(f"R² del modelo con selección hacia atrás: {r2_backward:.4f}")

print(f"La diferencia entre los modelos es de: {r2 - r2_backward:.4f}")


R² del modelo con selección hacia atrás: 0.3959
La diferencia entre los modelos es de: 0.0054


La diferencia entre los modelos es demasiado pequeña. Si queremos una regresión un poco más precisa aunque use más poder computacional, podemos usar la de hacia adelante. Pero si queremos ahorrarnos un poco de tiempo con prácticamente los mismos resultados, podemos usar la de hacia atras sin mucha penalización.
